In [14]:
clear all

In [15]:
import findspark

In [16]:
# find spark location

findspark.init('/home/chunyi/spark-2.2.1-bin-hadoop2.7')

In [17]:
import pyspark

In [18]:
from pyspark.sql import SparkSession

In [19]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [20]:
from pyspark.sql.types import (StructField, StringType,
                              IntegerType, StructType)

In [21]:
def stripSpace(string):
    return string.replace(' ','')

In [22]:
# load all data

data = spark.read.csv('../data/all_play_simple_withrate.log.fn', sep='\t')

In [23]:
from pyspark.sql.functions import udf

udf_stripSpace = udf(stripSpace, StringType())

In [24]:
# rename all columns

data = data.withColumn('user_id', udf_stripSpace(data['_c0']))
data = data.withColumn('song_id', udf_stripSpace(data['_c1']))
data = data.withColumn('play_times', udf_stripSpace(data['_c2']))
#data = data.withColumn('song_length', udf_stripSpace(data['_c3']))
#data = data.withColumn('paid', udf_stripSpace(data['_c4']))

In [25]:
data = data.select(data['user_id'].cast('int'),
                  data['song_id'].cast('int'),
                  data['play_times'].cast('int'))

In [27]:
# filter out null values 

data = data.filter('user_id > 0 and song_id > 0 and play_times >= 0 ')
#data.describe().show()

In [30]:
# to check how many songs each user played, the purpose is to check wheather active user or not
data.createOrReplaceTempView('data')
user_song_count = spark.sql("SELECT user_id, count(song_id) as Played_song FROM data GROUP BY user_id")
#user_song_count.show(5)

In [31]:
user_song_count.count()

Py4JJavaError: An error occurred while calling o296.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 4.0 failed 1 times, most recent failure: Lost task 3.0 in stage 4.0 (TID 10, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/chunyi/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/home/chunyi/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/chunyi/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 220, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/chunyi/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 138, in dump_stream
    for obj in iterator:
  File "/home/chunyi/spark-2.2.1-bin-hadoo

In [32]:
# make a dataframe of inactive users

inactive_user = user_song_count.filter('count(song_id) < 4')
inactive_user.count()

Py4JJavaError: An error occurred while calling o341.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 7.0 failed 1 times, most recent failure: Lost task 3.0 in stage 7.0 (TID 15, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/chunyi/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 177, in main
    process()
  File "/home/chunyi/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 172, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/chunyi/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 220, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/chunyi/spark-2.2.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 138, in dump_stream
    for obj in iterator:
  File "/home/chunyi/spark-2.2.1-bin-hadoo

In [352]:
data.show(5)

+---------+--------+-----------+-----------+----+
|  user_id| song_id|play_length|song_length|paid|
+---------+--------+-----------+-----------+----+
|154422682|20870993|      22013|        332|   0|
|154421907| 6560858|         96|        161|   0|
|154422630| 3385963|     235868|        235|   0|
|154410267| 6777172|        164|        237|   0|
|154422626| 3198036|     275249|          0|   0|
+---------+--------+-----------+-----------+----+
only showing top 5 rows



In [355]:
# calculate the most popular songs
popular_songs = spark.sql("SELECT song_id, count(user_id) as user_count FROM data WHERE play_length*2 > song_length GROUP BY song_id ORDER BY user_count DESC")
popular_songs.show(20)

+--------+----------+
| song_id|user_count|
+--------+----------+
|20870989|      1689|
|20870988|      1628|
|20870987|      1367|
|20870990|      1302|
|20870991|       973|
|20862199|       844|
|15249349|       687|
|20870992|       665|
|20870993|       521|
| 9950164|       517|
|15807836|       387|
| 6468891|       348|
| 5237384|       280|
|20868396|       254|
|20804585|       221|
|  466227|       214|
| 5114569|       193|
| 6411519|       174|
| 7176392|       150|
| 6749207|       139|
+--------+----------+
only showing top 20 rows



In [356]:
data.show(5)

+---------+--------+-----------+-----------+----+
|  user_id| song_id|play_length|song_length|paid|
+---------+--------+-----------+-----------+----+
|154422682|20870993|      22013|        332|   0|
|154421907| 6560858|         96|        161|   0|
|154422630| 3385963|     235868|        235|   0|
|154410267| 6777172|        164|        237|   0|
|154422626| 3198036|     275249|          0|   0|
+---------+--------+-----------+-----------+----+
only showing top 5 rows



In [357]:
# use max song length to replace original song length
max_song_length_matrix.createOrReplaceTempView('max_song_length_matrix')
data = spark.sql("SELECT data.user_id, data.song_id, data.play_length,  max_song_length_matrix.max_length AS song_length FROM data LEFT JOIN max_song_length_matrix ON data.song_id = max_song_length_matrix.song_id" )

In [358]:
data.describe().show()

+-------+--------------------+-----------------+------------------+-----------------+
|summary|             user_id|          song_id|       play_length|      song_length|
+-------+--------------------+-----------------+------------------+-----------------+
|  count|               93447|            93447|             93447|            93447|
|   mean|1.4542572819025758E8|7937701.708690488|15438.451860412855|270.7987950388991|
| stddev|3.3252280201555815E7|7264628.432150167| 63577.29571119192|333.8384470287966|
|    min|              168955|             1672|                 0|                0|
|    max|           154465966|         20876814|           2519470|             8946|
+-------+--------------------+-----------------+------------------+-----------------+



In [359]:
from pyspark.sql.functions import col, when, round, sum, avg
    

In [360]:
# calculate how many times using play_length and song_length column
data = data.withColumn('play_times', when(col('song_length') == 0, 0).otherwise(round(col('play_length')/col('song_length'))))

#data.describe().show(5)

In [361]:
# change column to int type
data = data.select(data['user_id'],
                  data['song_id'],
                  data['play_length'],
                  data['song_length'],
                  data['play_times'].cast('int'))

In [362]:
data.describe().show()

+-------+--------------------+-----------------+------------------+-----------------+------------------+
|summary|             user_id|          song_id|       play_length|      song_length|        play_times|
+-------+--------------------+-----------------+------------------+-----------------+------------------+
|  count|               93447|            93447|             93447|            93447|             93447|
|   mean|1.4542572819025758E8|7937701.708690488|15438.451860412855|270.7987950388991| 52.17460164585273|
| stddev|3.3252280201555815E7|7264628.432150167| 63577.29571119192|333.8384470287966|217.95430310006304|
|    min|              168955|             1672|                 0|                0|                 0|
|    max|           154465966|         20876814|           2519470|             8946|              1191|
+-------+--------------------+-----------------+------------------+-----------------+------------------+



In [363]:
data.createOrReplaceTempView('data')

In [364]:
# for each user, calculate how many times each song plays

data = spark.sql("SELECT user_id, song_id,  sum(play_times) AS total_play_times, sum(play_length) AS total_play_length, max(song_length) AS song_length FROM data GROUP BY user_id, song_id" )

In [365]:
data.show(5)

+---------+--------+----------------+-----------------+-----------+
|  user_id| song_id|total_play_times|total_play_length|song_length|
+---------+--------+----------------+-----------------+-----------+
|154420326|15208411|               0|               22|         59|
|154415167|  707143|               1|              228|        228|
|154420350| 6560373|               0|                1|        173|
|154422354| 7175320|               0|               11|        211|
|154408915| 6893789|               3|              285|         95|
+---------+--------+----------------+-----------------+-----------+
only showing top 5 rows



In [366]:
data.describe().show()

+-------+--------------------+-----------------+-----------------+------------------+------------------+
|summary|             user_id|          song_id| total_play_times| total_play_length|       song_length|
+-------+--------------------+-----------------+-----------------+------------------+------------------+
|  count|               70629|            70629|            70629|             70629|             70629|
|   mean|1.4711416415766895E8|7940891.432102961|69.03056818020926|20426.128233445186| 275.7063104390548|
| stddev|2.9985219740135584E7|7414897.871879102|291.5746173380588| 98108.67151239264|340.99571065785665|
|    min|              168955|             1672|                0|                 0|                 0|
|    max|           154465966|         20876814|            27944|           8465072|              8946|
+-------+--------------------+-----------------+-----------------+------------------+------------------+



In [367]:
# based on total play times create ratings
import math
from pyspark.sql.functions import log
data = data.withColumn('rating', when(data["total_play_times"] >= 4, 5).otherwise((data["total_play_times"]  ).cast('int') ) )
#data = data.withColumn('rating', when(col('total_play_times') == 0, 0).otherwise(2*log(col('total_play_times')) ))

In [368]:
data.show(5)

+---------+--------+----------------+-----------------+-----------+------+
|  user_id| song_id|total_play_times|total_play_length|song_length|rating|
+---------+--------+----------------+-----------------+-----------+------+
|154420326|15208411|               0|               22|         59|     0|
|154415167|  707143|               1|              228|        228|     1|
|154420350| 6560373|               0|                1|        173|     0|
|154422354| 7175320|               0|               11|        211|     0|
|154408915| 6893789|               3|              285|         95|     3|
+---------+--------+----------------+-----------------+-----------+------+
only showing top 5 rows



In [369]:
data.describe().show()


+-------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|             user_id|          song_id| total_play_times| total_play_length|       song_length|            rating|
+-------+--------------------+-----------------+-----------------+------------------+------------------+------------------+
|  count|               70629|            70629|            70629|             70629|             70629|             70629|
|   mean|1.4711416415766895E8|7940891.432102961|69.03056818020926|20426.128233445186| 275.7063104390548|0.9596766200852341|
| stddev|2.9985219740135584E7|7414897.871879102|291.5746173380588| 98108.67151239264|340.99571065785665|1.3873531673183468|
|    min|              168955|             1672|                0|                 0|                 0|                 0|
|    max|           154465966|         20876814|            27944|           8465072|              8946|                 5|
+-------

In [370]:
### Recommender system ####
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [434]:
matrix_recommender = data.select('user_id', 'song_id', 'rating')
matrix_recommender = matrix_recommender.filter('rating > 1')

#del(data)

In [435]:
matrix_recommender.show(5)

+---------+--------+------+
|  user_id| song_id|rating|
+---------+--------+------+
|154408915| 6893789|     3|
|154407321| 1244660|     5|
|154422946|15807836|     5|
|154422828| 4223040|     2|
|154423513|20870988|     5|
+---------+--------+------+
only showing top 5 rows



In [436]:
(training, test) = matrix_recommender.randomSplit([0.8, 0.2])

In [437]:
als = ALS(maxIter=10, regParam=0.01, userCol="user_id", itemCol="song_id", ratingCol="rating")


In [438]:
#del(matrix_recommender)

In [439]:
training.describe().show()

+-------+--------------------+--------------------+------------------+
|summary|             user_id|             song_id|            rating|
+-------+--------------------+--------------------+------------------+
|  count|                7520|                7520|              7520|
|   mean|1.4818004088922873E8|1.0420211540558511E7| 4.052260638297873|
| stddev|2.8016794676028725E7|   8231998.164158167|1.3560493129727433|
|    min|              295224|                1756|                 2|
|    max|           154465778|            20876481|                 5|
+-------+--------------------+--------------------+------------------+



In [501]:
model = als.fit(matrix_recommender)



In [530]:
user_recs = model.recommendForAllUsers(3)


In [531]:
user_recs.show(5)

+---------+--------------------+
|  user_id|     recommendations|
+---------+--------------------+
|154463220|[[952877,5.786953...|
|154425034|[[1217815,14.4940...|
|154425484|[[6855432,4.70724...|
|154425855|[[3602398,14.5501...|
|154412037|[[7190414,4.93858...|
+---------+--------------------+
only showing top 5 rows



In [532]:
user_song_count.show(5)

+---------+-----------+
|  user_id|Played_song|
+---------+-----------+
|154412037|         35|
|154417891|         10|
|154421973|         53|
|154409364|          6|
|154409339|         29|
+---------+-----------+
only showing top 5 rows



In [533]:
user_recs.createOrReplaceTempView('user_recs')
user_song_count.createOrReplaceTempView('user_song_count')

user_recs = spark.sql("SELECT user_id, recommendations[0].song_id AS first_choice, recommendations[1].song_id AS second_choice, recommendations[2].song_id AS third_choice FROM user_recs" )

In [534]:
user_recs.show(5)


+---------+------------+-------------+------------+
|  user_id|first_choice|second_choice|third_choice|
+---------+------------+-------------+------------+
|154463220|      952877|       203164|      703815|
|154425034|     1217815|       952877|      203164|
|154425484|     6855432|     11773482|     3849211|
|154425855|     3602398|      6308715|     1217815|
|154412037|     7190414|       280638|    15706616|
+---------+------------+-------------+------------+
only showing top 5 rows



In [535]:
user_recs.createOrReplaceTempView('user_recs')
user_recs = spark.sql("SELECT c.user_id, r.first_choice, r.second_choice, r.third_choice,  c.Played_song FROM user_recs r RIGHT JOIN user_song_count c ON r.user_id = c.user_id") 

In [536]:
user_recs.show(25)

+---------+------------+-------------+------------+-----------+
|  user_id|first_choice|second_choice|third_choice|Played_song|
+---------+------------+-------------+------------+-----------+
| 10226023|        null|         null|        null|          2|
|127185921|        null|         null|        null|         16|
|139502126|        null|         null|        null|         17|
|153952805|        null|         null|        null|         76|
|154409339|       96217|      7107690|     3514084|         29|
|154409364|        null|         null|        null|          6|
|154412037|     7190414|       280638|    15706616|         35|
|154417891|        null|         null|        null|         10|
|154419409|        null|         null|        null|        106|
|154421973|        null|         null|        null|         53|
|154423790|        null|         null|        null|         16|
|154424089|     6414906|      6635279|      650921|         12|
|154424244|        null|         null|  

In [537]:
pop_song1 = popular_songs.head(3)[0].song_id
pop_song2 = popular_songs.head(3)[1].song_id
pop_song3 = popular_songs.head(3)[2].song_id


In [538]:
user_recs.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- first_choice: integer (nullable = true)
 |-- second_choice: integer (nullable = true)
 |-- third_choice: integer (nullable = true)
 |-- Played_song: long (nullable = false)



In [539]:
#user_recs.createOrReplaceTempView('user_recs')
#user_recs = spark.sql("SELECT user_id,  (IFNULL(first_choice, pop_song1)) AS my_first_choice, (IFNULL(second_choice, pop_song2)) AS my_second_choice, (IFNULL(third_choice, pop_song3)) AS my_third_choice FROM user_recs" )

user_recs = user_recs.withColumn('first_rec', when(user_recs['first_choice'].isNull(), pop_song1).otherwise(user_recs['first_choice']))
user_recs = user_recs.withColumn('second_rec', when(user_recs['second_choice'].isNull(), pop_song2).otherwise(user_recs['second_choice']))
user_recs = user_recs.withColumn('third_rec', when(user_recs['third_choice'].isNull(), pop_song3).otherwise(user_recs['third_choice']))


In [540]:
user_recs.show(10)

+---------+------------+-------------+------------+-----------+---------+----------+---------+
|  user_id|first_choice|second_choice|third_choice|Played_song|first_rec|second_rec|third_rec|
+---------+------------+-------------+------------+-----------+---------+----------+---------+
| 10226023|        null|         null|        null|          2| 20870989|  20870988| 20870987|
|127185921|        null|         null|        null|         16| 20870989|  20870988| 20870987|
|139502126|        null|         null|        null|         17| 20870989|  20870988| 20870987|
|153952805|        null|         null|        null|         76| 20870989|  20870988| 20870987|
|154409339|       96217|      7107690|     3514084|         29|    96217|   7107690|  3514084|
|154409364|        null|         null|        null|          6| 20870989|  20870988| 20870987|
|154412037|     7190414|       280638|    15706616|         35|  7190414|    280638| 15706616|
|154417891|        null|         null|        null

In [543]:
user_recs = user_recs.select(user_recs['user_id'],
                 user_recs['first_rec'],
                 user_recs['second_rec'],
                 user_recs['third_rec'])

In [544]:
user_recs.show(10)

+---------+---------+----------+---------+
|  user_id|first_rec|second_rec|third_rec|
+---------+---------+----------+---------+
| 10226023| 20870989|  20870988| 20870987|
|127185921| 20870989|  20870988| 20870987|
|139502126| 20870989|  20870988| 20870987|
|153952805| 20870989|  20870988| 20870987|
|154409339|    96217|   7107690|  3514084|
|154409364| 20870989|  20870988| 20870987|
|154412037|  7190414|    280638| 15706616|
|154417891| 20870989|  20870988| 20870987|
|154419409| 20870989|  20870988| 20870987|
|154421973| 20870989|  20870988| 20870987|
+---------+---------+----------+---------+
only showing top 10 rows



In [ ]:
####################################################################################################################

In [468]:
popular_songs.show(5)

+--------+----------+
| song_id|user_count|
+--------+----------+
|20870989|      1689|
|20870988|      1628|
|20870987|      1367|
|20870990|      1302|
|20870991|       973|
+--------+----------+
only showing top 5 rows



In [482]:
popular_songs.head(3)[2].song_id

20870987

In [47]:
item_recs = model.recommendForAllItems(3)

In [48]:
item_recs.show(10)

+-------+--------------------+
|song_id|     recommendations|
+-------+--------------------+
|  69042|[[1749320,0.0], [...|
|  87724|[[1749320,0.0], [...|
| 104656|[[1749320,0.0], [...|
| 110904|[[154437261,2.845...|
| 118989|[[154408665,3.138...|
| 156365|[[154425546,5.087...|
| 165310|[[1749320,0.0], [...|
| 200878|[[154437692,6.948...|
| 214739|[[1749320,0.0], [...|
| 255362|[[154424853,4.771...|
+-------+--------------------+
only showing top 10 rows



In [46]:

predictions = model.transform(test)
predictions.show()

+---------+-------+------+------------+
|  user_id|song_id|rating|  prediction|
+---------+-------+------+------------+
|  1685126|  69042|     0|         0.0|
|154425373|  87724|     0|         0.0|
|154419409| 110904|     0|0.0100495145|
|154415157| 110904|     1|   1.0004137|
|154460124| 118989|     1|   0.9813532|
|154421730| 200878|     1|   0.9352329|
|154425458| 255362|     0|         0.0|
|154464207| 371006|     1|   0.9738073|
|154420887| 403518|     0|         0.0|
|154426419| 509175|     0|         0.0|
|154449892| 534327|     1|   0.9562116|
|154464695| 534327|     1|   0.9712486|
|154460774| 534327|     1|   0.9915261|
|154431425| 643423|     2|   1.9456455|
|154435336| 643423|     0|  0.10103321|
|154459435| 879366|     1|  0.97986615|
|154426003|1029443|     1|  0.98263645|
|  1685126|2301431|     0| 0.005492702|
|154426268|3324609|     1|     0.97586|
|154414800|3324609|     1|   1.0084307|
+---------+-------+------+------------+
only showing top 20 rows



In [47]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [48]:
rmse = evaluator.evaluate(predictions)

In [49]:
print('RMSE')
print(rmse)


RMSE
0.0891132370161574


In [50]:
single_user = test.filter(test['user_id']==154425373).select(['user_id', 'song_id'])

In [51]:
single_user.show()

+---------+-------+
|  user_id|song_id|
+---------+-------+
|154425373|4847647|
|154425373|  87724|
+---------+-------+



In [52]:
recommendations = model.transform(single_user)

In [53]:
recommendations.orderBy('Prediction', ascending=False).show()

+---------+-------+----------+
|  user_id|song_id|prediction|
+---------+-------+----------+
|154425373|4847647|0.99685335|
|154425373|  87724|       0.0|
+---------+-------+----------+



In [105]:
data.groupBy("user_id", "song_id").count().show(5)

+---------+--------+-----+
|  user_id| song_id|count|
+---------+--------+-----+
|154420326|15208411|    1|
|154415167|  707143|    1|
|154420350| 6560373|    1|
|154422354| 7175320|    1|
|154408915| 6893789|    3|
+---------+--------+-----+
only showing top 5 rows



In [106]:
matrix = data.groupBy("user_id", "song_id").count()


In [107]:
matrix.show()

+---------+--------+-----+
|  user_id| song_id|count|
+---------+--------+-----+
|154420326|15208411|    1|
|154415167|  707143|    1|
|154420350| 6560373|    1|
|154422354| 7175320|    1|
|154408915| 6893789|    3|
|154420736|  857978|    1|
|154422927| 1068698|    1|
|154407321| 1244660|   26|
|154422946|15807836|    1|
|154417195| 4652947|    1|
|154413919| 3194852|    1|
|154422346| 6253074|    2|
|154422346| 1522598|    2|
|154421525| 3534553|    1|
|154416855|  996642|    1|
|154414758| 7922570|    1|
|154418894|11059349|    1|
|154423104|  169744|    1|
|154422828| 4223040|    2|
|154422674|  158181|    1|
+---------+--------+-----+
only showing top 20 rows



In [108]:
data.groupBy("song_id").count().describe().show()

+-------+--------------------+------------------+
|summary|             song_id|             count|
+-------+--------------------+------------------+
|  count|               27250|             27251|
|   mean|   5033627.875853211|3.6695901067850722|
| stddev|1.1072267282073868E7| 44.91230884564563|
|    min|         -1649718862|                 1|
|    max|            20876814|              5474|
+-------+--------------------+------------------+



In [109]:
song_count = data.groupBy("song_id").count()

In [110]:
song_count.count()

27251

In [111]:
song_count.filter("count > 1").count()

10069

In [112]:
song_count.withColumn('popular_song',  song_count['count']).show(5)

+--------+-----+------------+
| song_id|count|popular_song|
+--------+-----+------------+
|20869416|   94|          94|
| 7112577|    6|           6|
|  462334|    6|           6|
| 2948915|    2|           2|
| 7065988|    3|           3|
+--------+-----+------------+
only showing top 5 rows



In [113]:
song_count = song_count.withColumnRenamed('count', 'total_play_count') 

In [114]:
song_count.show(5)

+--------+----------------+
| song_id|total_play_count|
+--------+----------------+
|20869416|              94|
| 7112577|               6|
|  462334|               6|
| 2948915|               2|
| 7065988|               3|
+--------+----------------+
only showing top 5 rows



In [115]:
user_count = data.groupBy('user_id').count()

In [116]:
user_count.show(5)

+---------+-----+
|  user_id|count|
+---------+-----+
|154412037|   35|
|154417891|   10|
|154421973|   56|
|154409364|    6|
|154409339|   29|
+---------+-----+
only showing top 5 rows



In [117]:
user_count = user_count.withColumnRenamed('count', 'total_user_count')


In [118]:
user_count.show(5)

+---------+----------------+
|  user_id|total_user_count|
+---------+----------------+
|154412037|              35|
|154417891|              10|
|154421973|              56|
|154409364|               6|
|154409339|              29|
+---------+----------------+
only showing top 5 rows



In [119]:
matrix.createOrReplaceTempView('data')
user_count.createOrReplaceTempView('user_count')

In [120]:
combine1 = spark.sql("SELECT data.user_id, data.song_id, data.count, user_count.total_user_count FROM data JOIN user_count ON data.user_id = user_count.user_id" )

In [121]:
combine1.count()

72039

In [31]:
combine1.show(5)

+---------+--------+-----+----------------+
|  user_id| song_id|count|total_user_count|
+---------+--------+-----+----------------+
|154420326|15208411|    1|              23|
|154415167|  707143|    1|              15|
|154420350| 6560373|    1|              96|
|154422354| 7175320|    1|               3|
|154408915| 6893789|    3|              46|
+---------+--------+-----+----------------+
only showing top 5 rows



In [122]:
song_count.createOrReplaceTempView('song_count')
combine1.createOrReplaceTempView('combine1')


In [136]:
combine2 = spark.sql("SELECT combine1.*,  song_count.total_play_count FROM combine1 JOIN song_count ON combine1.song_id = song_count.song_id" )

In [137]:
combine2.describe().show()

+-------+--------------------+-----------------+------------------+------------------+-----------------+
|summary|             user_id|          song_id|             count|  total_user_count| total_play_count|
+-------+--------------------+-----------------+------------------+------------------+-----------------+
|  count|               72004|            72004|             72004|             72004|            72004|
|   mean|1.4702823311017445E8|7846242.112965946|1.3859646686295206|108.93175379145603|  320.11788233987|
| stddev| 3.019756936019393E7| 9649417.46764295|3.3917304342656163| 380.1013408280859|772.9581538056328|
|    min|                   0|      -1649718862|                 1|                 1|                1|
|    max|           154465997|         20876814|               713|              2532|             5474|
+-------+--------------------+-----------------+------------------+------------------+-----------------+



In [35]:
combine2.show(5)

+---------+--------+-----+----------------+----------------+
|  user_id| song_id|count|total_user_count|total_play_count|
+---------+--------+-----+----------------+----------------+
|154420326|15208411|    1|              23|               1|
|154415167|  707143|    1|              15|               1|
|154420350| 6560373|    1|              96|               1|
|154422354| 7175320|    1|               3|               1|
|154408915| 6893789|    3|              46|              16|
+---------+--------+-----+----------------+----------------+
only showing top 5 rows



In [138]:
from pyspark.sql.functions import when

In [153]:
combine2 = combine2.withColumn('rating', when(combine2["count"] > 4, 5).otherwise((combine2["count"] ).cast('int') ) )

In [146]:
combine2.show(5)

+---------+--------+-----+----------------+----------------+------+
|  user_id| song_id|count|total_user_count|total_play_count|rating|
+---------+--------+-----+----------------+----------------+------+
|154420326|15208411|    1|              23|               1|     3|
|154415167|  707143|    1|              15|               1|     3|
|154420350| 6560373|    1|              96|               1|     3|
|154422354| 7175320|    1|               3|               1|     3|
|154408915| 6893789|    3|              46|              16|     5|
+---------+--------+-----+----------------+----------------+------+
only showing top 5 rows



In [147]:
combine2.printSchema()


root
 |-- user_id: integer (nullable = true)
 |-- song_id: integer (nullable = true)
 |-- count: long (nullable = false)
 |-- total_user_count: long (nullable = false)
 |-- total_play_count: long (nullable = false)
 |-- rating: integer (nullable = false)



In [148]:
### Recommender system ####
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [149]:
matrix_recommender = combine2.select('user_id', 'song_id', 'rating')


In [150]:
matrix_recommender.describe().show()

+-------+--------------------+-----------------+------------------+
|summary|             user_id|          song_id|            rating|
+-------+--------------------+-----------------+------------------+
|  count|               72004|            72004|             72004|
|   mean|1.4702823311017445E8|7846242.112965946|3.2295289150602744|
| stddev| 3.019756936019393E7| 9649417.46764295|0.5833875522146803|
|    min|                   0|      -1649718862|                 3|
|    max|           154465997|         20876814|                 6|
+-------+--------------------+-----------------+------------------+



In [151]:
(training, test) = matrix_recommender.randomSplit([0.8, 0.2])

In [152]:
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="song_id", ratingCol="rating")
model = als.fit(training)

predictions = model.transform(test)
predictions.show()

+---------+-------+------+-----------+
|  user_id|song_id|rating| prediction|
+---------+-------+------+-----------+
|154460124| 118989|     3|-0.18767042|
|154420071| 156365|     3|  2.1347246|
|154458288| 156365|     3| 0.35034487|
|154430537| 214739|     3|        NaN|
|154437043| 255362|     3| 0.71764827|
|154415183| 255362|     3|  3.3322303|
| 37025504| 297391|     3|        NaN|
|154448042| 304915|     3|        NaN|
|154423858| 311014|     3|        NaN|
|154446542| 327733|     3|  2.8554392|
|154436860| 371006|     3|  0.9415884|
|154417014| 455910|     3|        NaN|
|154426419| 509175|     3|        NaN|
|154464695| 534327|     3|  3.8092153|
|154460944| 534327|     3| -2.7885003|
|154464310| 534327|     3| 0.35848787|
|154463366| 534327|     3|  2.9445949|
|154463517| 534327|     3|  1.0452793|
|154431425| 643423|     4| 0.46140617|
|154422868| 643423|     3| -1.1077935|
+---------+-------+------+-----------+
only showing top 20 rows

